In [2]:
import pandas as pd

In [12]:
# Put csv into df
df = pd.read_csv('data.csv')

In [9]:

# Separate features and label
X = df.drop('score', axis=1)
y = df['score']
mu = calculate_mu()
# Calculate mu
# Fit for mu
# Save weights and intercepts

In [16]:
'delta' in df.columns

False